# Predicting Song Popularity#

### The Study###
"The music industry has a well-developed market with a global annual revenue around $15 billion. The recording industry is highly competitive and is dominated by three big production companies which make up nearly 82% of the total annual album sales.

Artists are at the core of the music industry and record labels provide them with the necessary resources to sell their music on a large scale. A record label incurs numerous costs (studio recording, marketing, distribution, and touring) in exchange for a percentage of the profits from album sales, singles and concert tickets.

Unfortunately, the success of an artist's release is highly uncertain: a single may be extremely popular, resulting in widespread radio play and digital downloads, while another single may turn out quite unpopular, and therefore unprofitable.

Knowing the competitive nature of the recording industry, record labels face the fundamental decision problem of which musical releases to support to maximize their financial success.

How can we use analytics to predict the popularity of a song? In this assignment, we challenge ourselves to predict whether a song will reach a spot in the Top 10 of the Billboard Hot 100 Chart.

Taking an analytics approach, we aim to use information about a song's properties to predict its popularity. The dataset songs.csv consists of all songs which made it to the Top 10 of the Billboard Hot 100 Chart from 1990-2010 plus a sample of additional songs that didn't make the Top 10. This data comes from three sources: Wikipedia, Billboard.com, and EchoNest.

The variables included in the dataset either describe the artist or the song, or they are associated with the following song attributes: time signature, loudness, key, pitch, tempo, and timbre."

(source: MITx)

### The Variables###
The variables included in the dataset either describe the artist or the song, or they are associated with the following song attributes: time signature, loudness, key, pitch, tempo, and timbre.

Here's a detailed description of the variables:

***year*** = the year the song was released

***songtitle*** = the title of the song

***artistname*** = the name of the artist of the song

***songID and artistID*** = identifying variables for the song and artist

***timesignature and timesignature_confidence*** = a variable estimating the time signature of the song, and the confidence in the estimate

***loudness*** = a continuous variable indicating the average amplitude of the audio in decibels

***tempo and tempo_confidence*** = a variable indicating the estimated beats per minute of the song, and the confidence in the estimate

***key and key_confidence*** = a variable with twelve levels indicating the estimated key of the song (C, C#, . . ., B), and the confidence in the estimate

***energy*** = a variable that represents the overall acoustic energy of the song, using a mix of features such as loudness

***pitch*** = a continuous variable that indicates the pitch of the song

***timbre_0_min, timbre_0_max, timbre_1_min, timbre_1_max, . . . , timbre_11_min, and timbre_11_max*** = variables that indicate the minimum/maximum values over all segments for each of the twelve values in the timbre vector (resulting in 24 continuous variables)

***Top10*** = a binary variable indicating whether or not the song made it to the Top 10 of the Billboard Hot 100 Chart (1 if it was in the top 10, and 0 if it was not)

In [1]:
songs = read.csv('songs.csv')

### Exploratory Data Analysis###

In [2]:
str(songs)

'data.frame':	7574 obs. of  39 variables:
 $ year                    : int  2010 2010 2010 2010 2010 2010 2010 2010 2010 2010 ...
 $ songtitle               : Factor w/ 7141 levels "̈́ l'or_e des bois",..: 6204 5522 241 3115 48 608 255 4419 2886 6756 ...
 $ artistname              : Factor w/ 1032 levels "50 Cent","98 Degrees",..: 3 3 3 3 3 3 3 3 3 12 ...
 $ songID                  : Factor w/ 7549 levels "SOAACNI1315CD4AC42",..: 595 5439 5252 1716 3431 1020 1831 3964 6904 2473 ...
 $ artistID                : Factor w/ 1047 levels "AR00B1I1187FB433EB",..: 671 671 671 671 671 671 671 671 671 507 ...
 $ timesignature           : int  3 4 4 4 4 4 4 4 4 4 ...
 $ timesignature_confidence: num  0.853 1 1 1 0.788 1 0.968 0.861 0.622 0.938 ...
 $ loudness                : num  -4.26 -4.05 -3.57 -3.81 -4.71 ...
 $ tempo                   : num  91.5 140 160.5 97.5 140.1 ...
 $ tempo_confidence        : num  0.953 0.921 0.489 0.794 0.286 0.347 0.273 0.83 0.018 0.929 ...
 $ key                   

In [3]:
summary(songs)

      year          songtitle              artistname  
 Min.   :1990   Intro    :  15   Various artists: 162  
 1st Qu.:1997   Forever  :   8   Anal Cunt      :  49  
 Median :2002   Home     :   7   Various Artists:  44  
 Mean   :2001   Goodbye  :   6   Tori Amos      :  41  
 3rd Qu.:2006   Again    :   5   Eels           :  37  
 Max.   :2010   Beautiful:   5   Napalm Death   :  37  
                (Other)  :7528   (Other)        :7204  
                songID                   artistID    timesignature  
 SOALSZJ1370F1A7C75:   2   ARAGWS81187FB3F768: 222   Min.   :0.000  
 SOANPAC13936E0B640:   2   ARL14X91187FB4CF14:  49   1st Qu.:4.000  
 SOBDGMX12B0B80808E:   2   AR4KS8C1187FB4CF3D:  41   Median :4.000  
 SOBUDCZ12A58A80013:   2   AR0JZZ01187B9B2C99:  37   Mean   :3.894  
 SODFRLK13134387FB5:   2   ARZGTK71187B9AC7F5:  37   3rd Qu.:4.000  
 SOEJPOK12A6D4FAFE4:   2   AR95XYH1187FB53951:  31   Max.   :7.000  
 (Other)           :7562   (Other)           :7157                  


Our data set ends in 2010. It might be interesting to find out how many songs came out that year.

In [14]:
songs_2010 = table(songs$year==2010)[[2]]
cat(paste(songs_2010, "songs were released in 2010."))

373 songs were released in 2010.

Michael Jackson was a prolific, and popular, song writer. It might be instructive to see how many songs of his appear in our data set.  

In [16]:
songs_MJ = table(songs$artistname=="Michael Jackson")[[2]]
cat(paste(songs_MJ, "songs were released by Michael Jackson."))

18 songs were released by Michael Jackson.

As our study concerns song popularity, a more interesting query would be which songs, if any, by Michael Jackson made the top 10. 

In [37]:
MJ_subset = subset(songs, artistname=="Michael Jackson")
songs_10 = MJ_subset[c("songtitle", "Top10")]
songs_10

,songtitle,Top10
4329,You Rock My World,1
6205,She's Out of My Life,0
6206,Wanna Be Startin' Somethin',0
6207,You Are Not Alone,1
6208,Billie Jean,0
6209,The Way You Make Me Feel,0
6210,Black or White,1
6211,Rock with You,0
6212,Bad,0
6213,I Just Can't Stop Loving You,0


It looks like Michael Jackson had 5 songs in the Top 10 over the period covered by our data set.

We find it interesting that our data set contains time signature as a variable. This is a musical trait considered by all who play in an ensemble, but rarely by those who simply listen to music. If this variable affects song popularity, it would be at a subconscious level, which would be fascinating.

In [41]:
table(songs$timesignature)


   0    1    3    4    5    7 
  10  143  503 6787  112   19 

As shown above, our 'timesignature' variable takes on the following values: 0,1,3,4,5,7. Four is the most popular signature. 

Sometimes one is simply in the mood to hit the dance floor and throw down. When this is the case, it's best to have a DJ spinning a song with a fast tempo (measured in beats per minute). 

In [58]:
fastest_song=songs$songtitle[which.max(songs$tempo)]
cat(paste("The song with the fastest tempo is", fastest_song))

The song with the fastest tempo is Wanna Be Startin' Somethin'

# The Model##

We'll build a logistic model, as our response variable, Top10, represents a binary classification. The model will include, however, only those variables that describe the numerical properties of the song.

In [66]:
train = subset(songs, year <= 2009)
test = subset(songs, year == 2010)

In [67]:
exclusions = c('year', 'songtitle', 'artistname', 'songID', 'artistID')
train2 = train[,!(names(train)%in% exclusions)]
test2 = test[,!(names(test) %in% exclusions)]

In [68]:
str(test2)

'data.frame':	373 obs. of  34 variables:
 $ timesignature           : int  3 4 4 4 4 4 4 4 4 4 ...
 $ timesignature_confidence: num  0.853 1 1 1 0.788 1 0.968 0.861 0.622 0.938 ...
 $ loudness                : num  -4.26 -4.05 -3.57 -3.81 -4.71 ...
 $ tempo                   : num  91.5 140 160.5 97.5 140.1 ...
 $ tempo_confidence        : num  0.953 0.921 0.489 0.794 0.286 0.347 0.273 0.83 0.018 0.929 ...
 $ key                     : int  11 10 2 1 6 4 10 5 9 11 ...
 $ key_confidence          : num  0.453 0.469 0.209 0.632 0.483 0.627 0.715 0.423 0.751 0.602 ...
 $ energy                  : num  0.967 0.985 0.99 0.939 0.988 ...
 $ pitch                   : num  0.024 0.025 0.026 0.013 0.063 0.038 0.026 0.033 0.027 0.004 ...
 $ timbre_0_min            : num  0.002 0 0.003 0 0 ...
 $ timbre_0_max            : num  57.3 57.4 57.4 57.8 56.9 ...
 $ timbre_1_min            : num  -6.5 -37.4 -17.2 -32.1 -223.9 ...
 $ timbre_1_max            : num  171 171 171 221 171 ...
 $ timbre_2_min     

In [69]:
logistic1 = glm(Top10 ~ ., data=train2, family=binomial)

In [70]:
summary(logistic1)


Call:
glm(formula = Top10 ~ ., family = binomial, data = train2)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9220  -0.5399  -0.3459  -0.1845   3.0770  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)               1.470e+01  1.806e+00   8.138 4.03e-16 ***
timesignature             1.264e-01  8.674e-02   1.457 0.145050    
timesignature_confidence  7.450e-01  1.953e-01   3.815 0.000136 ***
loudness                  2.999e-01  2.917e-02  10.282  < 2e-16 ***
tempo                     3.634e-04  1.691e-03   0.215 0.829889    
tempo_confidence          4.732e-01  1.422e-01   3.329 0.000873 ***
key                       1.588e-02  1.039e-02   1.529 0.126349    
key_confidence            3.087e-01  1.412e-01   2.187 0.028760 *  
energy                   -1.502e+00  3.099e-01  -4.847 1.25e-06 ***
pitch                    -4.491e+01  6.835e+00  -6.570 5.02e-11 ***
timbre_0_min              2.316e-02  4.256e-03   5.441 5.

***Initial Observations:***
- Higher key_confidence, timesignature_confidence, & tempo_confidence leads to a higher likelihood a song will land in the Top 10.
- Lower key_confidence, timesignature_confidence, & tempo_confidence is the hallmark of songs that are less complex. The coefficients on these variables are low, so the typical mainstream song preferred by listeners most likely lacks complexity.
- Songs with heavy instrumentation tend to be loud and often energetic.  The coefficient for loudness supports the notion that listeners enjoy this trait, however, the coefficient for energy does not. Multicollinearity could be at play. 

In [79]:
correl = cor(train2$loudness, train2$energy)
cat(paste("The correlation between 'loudness' & 'energy':", round(correl,4)))

The correlation between 'loudness' & 'energy': 0.7399

Multicollinearity is indeed at work within our model and either loudness or energy must go. We will try eliminating loudness simply because loud songs are not necessarily good songs. 

In [80]:
logistic2 = glm(Top10 ~ . -loudness, data=train2, family=binomial)

In [81]:
summary(logistic2)


Call:
glm(formula = Top10 ~ . - loudness, family = binomial, data = train2)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.0983  -0.5607  -0.3602  -0.1902   3.3107  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.241e+00  7.465e-01  -3.002 0.002686 ** 
timesignature             1.625e-01  8.734e-02   1.860 0.062873 .  
timesignature_confidence  6.885e-01  1.924e-01   3.578 0.000346 ***
tempo                     5.521e-04  1.665e-03   0.332 0.740226    
tempo_confidence          5.497e-01  1.407e-01   3.906 9.40e-05 ***
key                       1.740e-02  1.026e-02   1.697 0.089740 .  
key_confidence            2.954e-01  1.394e-01   2.118 0.034163 *  
energy                    1.813e-01  2.608e-01   0.695 0.486991    
pitch                    -5.150e+01  6.857e+00  -7.511 5.87e-14 ***
timbre_0_min              2.479e-02  4.240e-03   5.847 5.01e-09 ***
timbre_0_max             -1.007e-01  1.178e-02

The removal of our 'loudness' variable from the model swung the coefficient for 'energy' in the right direction, however, it is no longer significant. Moreover, our AIC increased from 4827.2 to 4937.8. This indicates we now have an inferior model. We'll now see what happens when we keep 'loudness' in our model and eliminate 'energy'. 

In [82]:
logistic3 = glm(Top10 ~ . -energy, data=train2, family=binomial)

In [83]:
summary(logistic3)


Call:
glm(formula = Top10 ~ . - energy, family = binomial, data = train2)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9182  -0.5417  -0.3481  -0.1874   3.4171  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)               1.196e+01  1.714e+00   6.977 3.01e-12 ***
timesignature             1.151e-01  8.726e-02   1.319 0.187183    
timesignature_confidence  7.143e-01  1.946e-01   3.670 0.000242 ***
loudness                  2.306e-01  2.528e-02   9.120  < 2e-16 ***
tempo                    -6.460e-04  1.665e-03  -0.388 0.698107    
tempo_confidence          3.841e-01  1.398e-01   2.747 0.006019 ** 
key                       1.649e-02  1.035e-02   1.593 0.111056    
key_confidence            3.394e-01  1.409e-01   2.409 0.015984 *  
pitch                    -5.328e+01  6.733e+00  -7.914 2.49e-15 ***
timbre_0_min              2.205e-02  4.239e-03   5.200 1.99e-07 ***
timbre_0_max             -3.105e-01  2.537e-02 -

While our AIC increased marginally from 4827.2 to 4848.7, indicating a slight decline in model quality, removing the 'energy' term from the model eliminated our multicollinearity issue and left us with our highly significant 'loudness' variable. Thus, the slight increase in AIC seems worth it. 

### Out-of-Sample Testing ###

In [90]:
logistic3_pred = predict(logistic3, newdata=test2, type='response')

**Confusion Matrix Calculation**

In [91]:
confusion_mtx = function(df, observed_y, predicted_y, threshold) {
    
    # Input: dataframe, response variable name as string, vector of 
    # predicted values, threshold value
    # 
    # Output: Confusion matrix to assess model performance
    
    mtx=table(df[[observed_y]], predicted_y > threshold)
    
    return (cmtx)
}


In [94]:
cmtx=confusion_mtx(test2, 'Top10', logistic3_pred, .45)
addmargins(cmtx)

,FALSE,TRUE,Sum
0,309,5,314
1,40,19,59
Sum,349,24,373


**Confusion Matrix Calculation**

In [95]:
confusion_accuracy = function(confusion_matrix) {
    
    # Input: confusion matrix
    #
    # Output: Message stating the accuracy of the model based on the data
    # contained in the matrix.
    
    accuracy = sum(diag(prop.table(confusion_matrix)))
    
    return (print(paste("Based on the confusion matrix, the accuracy of the model is ", round(accuracy,4)*100, "%", sep="")))
}

confusion_accuracy(cmtx)

[1] "Based on the confusion matrix, the accuracy of the model is 87.94%"


As the data above indicate, our out-of-sample model has an accuracy rate of 87.94%. This seems like a decent result, but how would it compare to a baseline prediction using the test set.

**Comparison to Baseline Model**

In [99]:
cat(paste(round(table(test2$Top10)[[1]]/(table(test2$Top10)[[1]]+table(test2$Top10)[[2]]),4)*100),"%")

84.18 %

Our baseline model would have an accuracy rate of 84.18%, so we have definitely taken a step in the right direction by building our model, but was it worth it?